In [34]:
import pandas as pd
import os

In [35]:
df = pd.read_csv(r'C:\Users\VinodKumarM\Desktop\TAX_DW_TABLE_CHANGES.csv',index_col=False)

In [36]:
dfseries = df.groupby(['TABLE_NAME'])['REQUIRED_COLUMNS'].apply(list)
path = 'C:\\Users\\VinodKumarM\\Desktop\\TABLE_DDLS\\'

In [37]:
for index, value in dfseries.items():
    columns = ",".join(value)
    ddl = "create or replace table `helix-data-dev.helix_fin_tax_dw."+index+"`("+columns+");"
    with open(os.path.join(path, 'gbcc_new', index + '.sql'), 'w') as file:
        file.write(ddl)
    

In [16]:
df['ATTRIBUTE19'] = None
df['ATTRIBUTE20'] = None
df['GSA_INDICATOR_FLAG'] = None

In [12]:
df = df.loc[:,['BUSINESS_GROUP_ID',
'ORGANIZATION_ID',
'NAME',
'DATE_FROM',
'DATE_TO',
'SHORT_CODE',
'SET_OF_BOOKS_ID',
'DEFAULT_LEGAL_CONTEXT_ID',
'RUN_DATE',
'KEY']]

In [13]:
df.to_csv(r'C:\Users\VinodKumarM\OneDrive - EQUINIX\Projects\bip_integration\Schema\HR_OPERATING_UNITS_DEV11.csv',index=None)

In [25]:
command = "data_tasks_json --module GL"

In [3]:
options = command.split()

In [13]:
if options[1]=='--module':
    module = options[2]
    tables = None
elif options[1]=='--tables':
    module = None
    tables = options[2]

In [15]:
tables

In [22]:
task_type = options[0]

In [23]:
if task_type in ['data_tasks_json', 'del_tasks_json']:
    print(True)

True


In [33]:
derive_where_clause_dag(command)

data_tasks_json
data_tasks_json


' where MODULE = "GL" '

In [32]:
def derive_where_clause_dag(command) -> str:
    options = command.split()
    if options[1]=='--module':
        module = options[2]
        tables = None
    elif options[1]=='--tables':
        module = None
        tables = options[2]
    
    task_type=options[0]
    print(options[0])
    print(task_type)
    if task_type in ['data_tasks_json', 'del_tasks_json']:
        if module is None:
            tables = '","'.join(tables.split(','))
            tables = '"' + tables + '"'
            where_clause = ' where TABLE_ID in ({})'.format(tables)
        else:
            where_clause = ' where MODULE = "{}" '.format(module)

    if task_type in ['tranformation_tasks_json']:
        if module is None:
            tables = '","'.join(tables.split(','))
            tables = '"' + tables + '"'
            where_clause = ' where TABLE_ID in ({}) and DW_TABLE_NAME is not null'.format(tables)
        else:
            where_clause = ' where MODULE = "{}" and DW_TABLE_NAME is not null'.format(module)
    
    return  where_clause